<h6><center>Big Data Algorithms Techniques & Platforms</center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Assignment 1: Introduction to MapReduce</center>
<hr style=" border:none; height:3px;">
</h1>


# <strong> Important </strong>

For this assignment, as you did in the first lab, do not use the native fuctions <code>map()</code> and <code>reduce()</code> of Python.


<hr>



## Flights and Airports Data

For this assignment you are going to analyze a dataset that will include flight data.

The dataset comes from <a href="https://www.kaggle.com/flashgordon/usa-airport-dataset">Kaggle</a> and is presented here as an extract called <code>extract_flights.csv</code>

The dataset is in a <code>.csv</code> file. Each line of the file represents a different flight. The data collected contain:


<code>Origin_airport</code>: Three letter airport code of the origin airport </br>
<code>Destination_airport</code>: Three letter airport code of the destination airport</br>
<code>Origin_city</code>: Origin city name</br>
<code>Destination_city</code>: Destination city name</br>
<code>Passengers</code>: Number of passengers transported from origin to destination</br>
<code>Seats</code>: Number of seats available on flights from origin to destination</br>
<code>Flights</code>: Number of flights between origin and destination (multiple records for one month, many with flights > 1)</br>
<code>Distance</code>: Distance (to nearest mile) flown between origin and destination</br>
<code>Fly_date</code>: The date (yyyymm) of flight</br>
<code>Origin_population</code>: Origin city's population as reported by US Census</br>
<code>Destination_population</code>: Destination city's population as reported by US Census</br>


Two files are available in the folder:

<ul>
  <li> <code> small_file.csv</code> </li>
  <li> <code> medium_file.csv</code> </li>
</ul>

For more complex analysis/statistics you can also download the original file available on kaggle.



### Helper function ... just in case

Here is a function that you can modify, reuse, not-use for giving you an idea about how to read a <code>.csv</code> file and remove the header.

In [153]:
import pandas as pd

def readCsv(filename):
  df = pd.read_csv (filename)
  #clean df
  #clear alter 0s
  df_list=df.values.tolist()
  return df_list

### <strong> Exercise 1 - Almost-empty flights</strong> 
#### <strong> 4 points </strong>
Describe and define a MapReduce procedure that gives as output the number of flights that departed with 10% of capacity maximum (i.e. empty, 1%, 6%, etc.) . 

The output can be of any form you like. 

You can use any data structure you want to support your implementation.




In [154]:
import string
from collections import defaultdict

#The map operation creates key-value pairs of the form ('flight capacity', 1) 
#The operation does not take into account flights that have 0 seats as these are not commercial flights

def map_e_flights(input):
  capacities=[]
  for line in input:
    if (line[5] != 0): #division by 0, if # of seats is 0 then do not consider
      #get flight capacity for all flights                 
      capacities.append(((float(line[4]/line[5])), line[6]))
  return capacities

#Shuffle function orders the criteria in a dictionary by grouping according to the flight capacity in this case
def shuffle_e_flights(input):
  dict_pairs=defaultdict(list)
  for index in input:
      dict_pairs[index[0]].append(index[1])
  return dict_pairs

#The reduce function sums up the number of instance for each flight capacity (key of the dictionary)
#It will add up the content of each key (i.e. the number of flights) if their capacity is below a certain threshold, here 10%
def reduce_e_flights(dict):
  res=0
  for i in dict:
    if(i<=0.1):
      res+=sum(dict[i])
  return res

answer=reduce_e_flights(shuffle_e_flights(map_e_flights(readCsv('small_file.csv'))))
print("The total number of flights under 10% capacity is", answer )

The total number of flights under 10% capacity is 1267


Look at your implementation and think about your program. Is there any option to add a `combine` operation?  If yes describe your new approach and provide an improved version of your code.

Describe in detail your procedure before the code (in the comment lines).

In [155]:
#We could combine the dictionary into sums instead of counting all the ones at first 
def partition_e_flights(input, N):
  return list((input[i:i+N] for i in range(0, len(input), N)))

def map_ep_flights(input):
  result=[]
  for key in input:
    #Inside each partition, perform the map function 
    result.append(map_e_flights(key))
  return result

def combine_ep_flights(input):
  combo=[]
  reduce_results=[]
  count=0
  #shuffle and reduce inside each partition
  for key in input:
    combo.append(shuffle_e_flights(key))
  for key in combo:
    count+=1
    reduce_results.append((count ,reduce_e_flights(key)))
  return reduce_results

#Shuffle - here break down all silos 
def shuffle_ep_flights(input):
  d = defaultdict(list)
  for x in input:
    d[x[0]].append(x[1])
  return d

#Reduce function
def reduce_ep_flights(dict):
  res=0
  for key in dict:
    res += sum(dict[key])
  return res

#N here is the number of partitions we will do - N should be updated accordingly to the size of the db 
N=100
answer= reduce_ep_flights(shuffle_ep_flights(combine_ep_flights(map_ep_flights(partition_e_flights(readCsv('small_file.csv'), N)))))
print("The total number of flights under 10% capacity is", answer )

The total number of flights under 10% capacity is 1267


### <strong> Exercise 2 - Top five destination airports </strong>
### <strong> 4 points </strong>

Provide now a function that lists the top five destination <strong>airports</strong>: the ones that have the highest number of incoming flights. Implement an algorithm that uses the MapReduce procedure.



In [156]:
#Map destination to incoming flights for each airport
#Destination list with incoming flights - return (destination, incoming flights)
def map_top_flights(input):
  res=[]
  for flight in input:
    res.append((flight[1], flight[6]))
  return res

#Here we order with the help of the shuffle function according to the DESTINATION Aiport acronym 
#We get a view of all flights #s departing from each airport
def shuffle_top_flights(input):
  d=defaultdict(list)
  for x in input:
    d[x[0]].append(x[1]) 
  return d


#In reduce, we add up all content of the keys of the dictionary
#And we filter out to pick the top 5 flights
#Filter returns the top 5 destinations and the total number of flights they have hosted in a tuple
def reduce_top_flights(dict):
  res=[]
  results=[]
  for key in dict:
    res.append((key, sum(dict[key])))
  results= sorted([(x[0],x[1]) for (i,x) in enumerate(res)], reverse=True )[:5]
  return results
 
answer=reduce_top_flights(shuffle_top_flights(map_top_flights(readCsv('small_file.csv'))))
print('The top five destination airports are ', answer)

The top five destination airports are  [('WDG', 3295), ('RDM', 108593), ('ERI', 7101), ('END', 7), ('EKO', 25742)]


### <strong> Exercise 3 - Top 5 destination cities </strong>
#### <strong>  2 points </strong>

Try to reuse the code you run before and define a function that lists the top five destination <strong>cities</strong>: the ones that have the highest number of incoming flights. Implement an algorithm that uses the MapReduce procedure.


In [166]:
#Map top five destination cities to incoming flights for each airport
#Destination cities with incoming flights - return (destination, incoming flights)
def map_top_cities(input):
  res=[]
  for flight in input:
    res.append((flight[3], flight[6]))
  return res

#Provide here few comment lines that describe your shuffle function

def shuffle_top_cities(input):
  d=shuffle_top_flights(input)
  return d

#Provide here few comment lines that describe your reduce function

def reduce_top_cities(dict):
  results=reduce_top_flights(dict)
  return results


answer=reduce_top_cities(shuffle_top_cities(map_top_cities(readCsv('small_file.csv'))))
print('The top five destination cities are: \n', answer)

The top five destination cities are: 
 [('Erie, PA', 7101), ('Enid, OK', 3302), ('Elko, NV', 25742), ('Bend, OR', 108593), ('Ames, IA', 1)]


## Statistics on flights

<p align="justify">
<font size="3">
Now we want to run some more complex analysis on the flights. 
</font>
</p>





### <strong> Exercise 4 - Top five connections by month</strong>
#### <strong> 4 points </strong>

Try to reuse the code you run before and define now a function that lists the top five connections by each month: the top five pairs of cities that have the most number of connections (the function takes into account the flights from A to B and from B to A by month/year). Implement an algorithm that uses the MapReduce procedure.

In [158]:
#In this question, we assume that from A to B and B to A is different, and does not represent a connection - In this manner these are seperate flights and a connection is simply a fligth from A-B
#We will return the tuple= tuple(flight date, ((city A, cityB), Flights))
#It is also important to highlight that the question will return top five conneciton by each month by each year. This is because it is most interesting to analyze the evolution of connections over time, it will have more impact in our analysis. 

#Here we map the top connection cities per months/years
def map_top_connections(input):
  res=[]
  for x in input:
    #Change date into tuple with only month and year
    year,month,date=x[8].split('-')
    res.append(((year,month),((x[2],x[3]),x[6])))
  return res

#Here in the shuffle we create a dictionary of monthly-yearly values
#We must also be careful to keep count of each shuffle operation within the dates, meaning for each cities 
def shuffle_top_connections(input):
  d= shuffle_top_flights(input)
  temp= defaultdict(list)
  result=[]
  f= defaultdict(list)
  for key in d:
    temp=shuffle_top_flights(d[key])
    for keyt in temp:
      result.append((key, (keyt, temp[keyt])))
  f= shuffle_top_flights(result)
  return f

#Again, we assume (A,B) and (B,A) are seperate and do not account for the same connection
#In the following function we reduce 
#We do another shuffle over the cities and their number of flights
#Once ordered we then reduce those
def reduce_top_connections(dict):
  d= dict
  result=[]
  temp= defaultdict(list)
  f= defaultdict(list)
  for key in d:
    temp=shuffle_top_flights(d[key])
    #clean up the double list 
    for each in temp:
      temp[each]=temp[each][0]
    result.append((key, reduce_top_flights(temp)))
  return result

answer=reduce_top_connections(shuffle_top_connections(map_top_connections(readCsv('small_file.csv'))))
print("Find below the top five pairs of destination cities by month and year :\n", answer)

Find below the top five pairs of destination cities by month and year :
 [(('2008', '10'), [(('Seattle, WA', 'Bend, OR'), 89), (('San Francisco, CA', 'Bend, OR'), 93), (('Salt Lake City, UT', 'Elko, NV'), 124), (('Salt Lake City, UT', 'Bend, OR'), 62), (('Portland, OR', 'Bend, OR'), 280)]), (('1990', '11'), [(('San Francisco, CA', 'Bend, OR'), 79), (('Portland, OR', 'Bend, OR'), 220), (('Pittsburgh, PA', 'Erie, PA'), 149), (('Medford, OR', 'Bend, OR'), 2), (('Klamath Falls, OR', 'Bend, OR'), 41)]), (('1990', '12'), [(('San Francisco, CA', 'Bend, OR'), 78), (('Portland, OR', 'Bend, OR'), 223), (('Pittsburgh, PA', 'Erie, PA'), 153), (('Klamath Falls, OR', 'Bend, OR'), 47), (('Eugene, OR', 'Bend, OR'), 19)]), (('1990', '10'), [(('San Francisco, CA', 'Bend, OR'), 85), (('Portland, OR', 'Bend, OR'), 230), (('Pittsburgh, PA', 'Erie, PA'), 152), (('Klamath Falls, OR', 'Bend, OR'), 50), (('Eugene, OR', 'Bend, OR'), 4)]), (('1990', '02'), [(('Worcester, MA', 'Erie, PA'), 1), (('Seattle, WA', 'B

Look at your code. Can you use a <code>combine()</code> operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

In [159]:
def partition_top_connections_r(input, N):
  return list((input[i:i+N] for i in range(0, len(input), N)))

#Again, a typical map function with the added for loop to ensure we loop inside each partition
def map_top_connections_r(input):
  result=[]
  for key in input:
    #Inside each partition, perform the map function 
    result.append(map_top_connections(key))
  return result

#The combine function loops within each partition
#Inside each partition, we perform a shuffle and a reduce
#Finally we return an array with our partitions reduced 
def combine_top_connections_r(input):
  shuffle_top=[]
  reduce_top=[]
  #Shuffle and get one dictionary per partition
  for partitions in input:
    shuffle_top.append(shuffle_top_connections(partitions))
  #Reduce each partition
  for parts in shuffle_top:
    reduce_top.append(reduce_top_connections(parts))
  return reduce_top

#We now must put all the data back together in the shuffle function
#We regroup everything in a list in order to be able to access the subsets of dates and perform shuffles the cities to get an absolute count
def shuffle_top_connections_r(input):
  #Variable declarations
  res=[]
  temp=[]
  temp_d= defaultdict(list)
  d=defaultdict(list)
  f=defaultdict(list)
  new_temp=[]
  
  #Create a list with all values, no more partitions
  for key in input:
    for date in key:
      temp.append((date[0], date[1]))
  #Shuffle operation on all dates 
  for x in temp:
    d[x[0]].append(x[1])
  for key in d:
    flat_list = [item for sublist in d[key] for item in sublist]
    temp_d=shuffle_top_flights(flat_list)
    #Shuffle operation on all cities 
    for x in temp_d:
      res.append((key, (x, temp_d[x])))
  f=shuffle_top_connections(res)
  return f

#Finally, we reduce everything again. Here the complexity lies in cleaning the data, for such I used a flattener function for the sake of clarity 
def reduce_top_connections_r(dict):
  result=[]
  clean_d= defaultdict(list)
  for key in dict:
    for x in dict[key]:
      flat_list = [item for sublist in x[1] for item in sublist]
      clean_d[key].append((x[0],flat_list))
  result=reduce_top_connections(clean_d)
  return result

#N below is the number of partitions we will do - N should be updated accordingly to the size of the db 
N=100
reduce_top_connections_r(shuffle_top_connections_r(combine_top_connections_r(map_top_connections_r(partition_top_connections_r(readCsv('small_file.csv'),N)))))

[(('2008', '10'),
  [(('Seattle, WA', 'Bend, OR'), 89),
   (('San Francisco, CA', 'Bend, OR'), 93),
   (('Salt Lake City, UT', 'Elko, NV'), 124),
   (('Salt Lake City, UT', 'Bend, OR'), 62),
   (('Portland, OR', 'Bend, OR'), 280)]),
 (('1990', '11'),
  [(('San Francisco, CA', 'Bend, OR'), 79),
   (('Portland, OR', 'Bend, OR'), 220),
   (('Pittsburgh, PA', 'Erie, PA'), 149),
   (('Medford, OR', 'Bend, OR'), 2),
   (('Klamath Falls, OR', 'Bend, OR'), 41)]),
 (('1990', '12'),
  [(('San Francisco, CA', 'Bend, OR'), 78),
   (('Portland, OR', 'Bend, OR'), 223),
   (('Pittsburgh, PA', 'Erie, PA'), 153),
   (('Klamath Falls, OR', 'Bend, OR'), 47),
   (('Eugene, OR', 'Bend, OR'), 19)]),
 (('1990', '10'),
  [(('San Francisco, CA', 'Bend, OR'), 85),
   (('Portland, OR', 'Bend, OR'), 230),
   (('Pittsburgh, PA', 'Erie, PA'), 152),
   (('Klamath Falls, OR', 'Bend, OR'), 50),
   (('Eugene, OR', 'Bend, OR'), 4)]),
 (('1990', '02'),
  [(('Worcester, MA', 'Erie, PA'), 1),
   (('Seattle, WA', 'Bend, OR'

### <strong> Exercise 5 - Number of full flights</strong>
#### <strong> 2 points </strong>
<p align="justify">
<font size="3">
Describe and implement an algorithm that, following MapReduce procedure, shows how many full flights have departed. This exercise gives you an idea about how many times you can re-use code in MapReduce with minimum effort for repetitive analysis.
</font>
</p>



In [160]:
#In the map function we first filter through full flights by checking whether the number of seats == number of passengers
#Here again it is important to note that 0 seats flights are not considered 
#We then return a tuple (ORIGIN AIRPORT, NUMBER OF FULL FLIGHTS)
def  map_full_flights(input):
  res =[]
  for flight in input:
    if(flight[5] !=0):
      if(flight[4]==flight[5]):
        res.append((flight[0], flight[6]))
  return res

#Create a dict of full flights with as key the AIRPORT acronym containing a # representing each full flight 
def shuffle_full_flights(input):
  d = shuffle_top_flights(input)
  return d

#In our reduce function here, we add up number of full flights using the dictionary's keys 
def reduce_full_flights(dict):
  result=0
  for i in dict:
      result+=sum(dict[i])
  return result

answer=reduce_full_flights(shuffle_full_flights(map_full_flights(readCsv('small_file.csv'))))
print('The number of full flights is', answer)


The number of full flights is 554



### <strong> Exercise 6 -  Average/percentage </strong>
#### <strong> 4 points </strong>

<p align="justify">
<font size="3">
Describe and implement a MapReduce procedure that gives, for each city, the percentage/average, of full flights that have departed.

Notice that this exercise shares some similarities with one of the previous exercises. Think how and if you can modify (generalize) one of the functions already implemented before. 
</font>
</p>



In [161]:
#We map cities to the # of departed flights and record the information on wheteher they were full or not
#We return (city, (full or not full) ,# departed flights)
#In the function below, if the flight is full we indicate such with a '1' - on the contrary, if the flight is not full, by a '0'
def map_average_full_flights(input):
  res=[]
  for flight in input:
    if(flight[5] != 0):
      if(flight[4]==flight[5]):
        res.append((flight[2], (1, flight[6])))
      else:
        res.append((flight[2], (0, flight[6])))
  return res

#(city, with all averages one after the other)
def shuffle_average_full_flights(input):
  d=defaultdict(list)
  d= shuffle_top_flights(input)
  return d 

#reduce one city, calculate the average 
def reduce_average_full_flights(dict):
  res=[]
  final_result=[]

  for key in dict:
    total_full=0
    total_non_full=0
    result_d=shuffle_full_flights(dict[key])
    #add up 0 key in dict
    total_non_full= (sum(result_d[0]))
    #add up 1 key in dict
    total_full= (sum(result_d[1]))
    #average both
    result_average= (total_full/(total_full+total_non_full))
    res.append((key, result_average))
  for x in res:
    #filter out 0s
    if(x[1]!=0):
      final_result.append((x[0],x[1]))
  return final_result

answer = reduce_average_full_flights(shuffle_average_full_flights(map_average_full_flights(readCsv('small_file.csv'))))
print('The percentage/average of full flights that have departed per city are found below: \n',answer)

The percentage/average of full flights that have departed per city are found below: 
 [('Eugene, OR', 0.0033444816053511705), ('Medford, OR', 0.01680672268907563), ('Seattle, WA', 0.00015600624024960998), ('Portland, OR', 1.5023361326863273e-05), ('Yakima, WA', 0.047619047619047616), ('Elko, NV', 0.022727272727272728), ('Salem, OR', 0.043478260869565216), ('Spokane, WA', 0.047058823529411764), ('Lewiston, ID', 0.06363636363636363), ('Port Angeles, WA', 0.06976744186046512), ('Longview, TX', 0.030303030303030304), ('Great Falls, MT', 0.05714285714285714), ('Fresno, CA', 0.012195121951219513), ('Tucson, AZ', 0.034482758620689655), ('Moses Lake, WA', 0.16666666666666666), ('Salt Lake City, UT', 0.00031113876789047915), ('Casper, WY', 0.027777777777777776), ('Reno, NV', 0.0003215434083601286), ('El Paso, TX', 0.01818181818181818), ('Albuquerque, NM', 0.03017241379310345), ('Fort Smith, AR', 0.08433734939759036), ('Wichita, KS', 0.03296703296703297), ('Duluth, MN', 0.1111111111111111), ('Ok

Look at your code. Can you use a <code>combine()</code> operation? 

If you cannot directly think about how you can represent the key-value pairs in a way that allows you to take advantage of a combine.

In [162]:
def partition_average_full_flights_r(input, N):
  return list((input[i:i+N] for i in range(0, len(input), N)))

#Again, a typical map function with the added for loop to ensure we loop inside each partition
def map_average_full_flights_r(input):
  res=[]
  for key in input:
    #Inside each partition, perform the map function
    res.append(map_average_full_flights(key))
  return res

#We take in all partitions and recombine them together 
#Here the trick lies in returning tuples with the total number of (full or not) flights that we recorded earlier
#We do this in order to be able to compute our average later in the reduce function, we need to bring forward the totals 
def combine_average_full_flights(input):
  shuffle_full=defaultdict(list)
  reduce_full=[]
  d= defaultdict(list)
  for parts in input:
    #inside each partition, shuffle
    shuffle_full=shuffle_average_full_flights(parts)
    #Reduce by adding all the 0s of each city and all the 1s back together - this will yield a total by city 
    for key, tup in shuffle_full.items():
      total_full=0
      total_non_full=0
      for x in tup: 
        if (x[0]==0):
          total_non_full += x[1]
        else:
          total_full += x[1]
      reduce_full.append((key, (total_full, total_non_full)))
  return reduce_full

#Filtering over empty partitions, we perform a normal shuffle
def shuffle_average_full_flights_r(input):
  d=defaultdict(list)
  temp=[]
  empty=[]
  for parts in input:
    if parts!=empty:
      temp.append(parts)
  d=shuffle_average_full_flights(temp)
  return d

#Here we reduce, however the averages must be kept clean 
def reduce_average_full_flights(dict):
  results=[]
  for key,x in dict.items():
      list_full_f= sum([count[0] for count in x])
      list_total_f=sum([count[1] for count in x])
      if (list_total_f==0):
        average=0
      else: 
        average=list_full_f/(list_total_f+list_full_f)
      results.append((key, average))
  return results

#N here is the number of partitions we will do - N should be updated accordingly to the size of the db 
N=100
answer = reduce_average_full_flights(shuffle_average_full_flights_r(combine_average_full_flights(map_average_full_flights_r(partition_average_full_flights_r(readCsv('small_file.csv'), N)))))
print('The percentage/average of full flights that have departed per city are found below: \n',answer)

The percentage/average of full flights that have departed per city are found below: 
 [('Manhattan, KS', 0.0), ('Eugene, OR', 0.0033444816053511705), ('Medford, OR', 0.01680672268907563), ('Seattle, WA', 0.00015600624024960998), ('Portland, OR', 1.5023361326863273e-05), ('Klamath Falls, OR', 0.0), ('San Francisco, CA', 0.0), ('Los Angeles, CA', 0.0), ('Wenatchee, WA', 0.0), ('Yakima, WA', 0.047619047619047616), ('Elko, NV', 0.022727272727272728), ('Salem, OR', 0.043478260869565216), ('Spokane, WA', 0.047058823529411764), ('Redding, CA', 0.0), ('Lewiston, ID', 0.06363636363636363), ('Astoria, OR', 0.0), ('Port Angeles, WA', 0.06976744186046512), ('Pendleton, OR', 0.0), ('San Jose, CA', 0.0), ('Eureka, CA', 0.0), ('Pullman, WA', 0.0), ('Sacramento, CA', 0.0), ('Fort Lauderdale, FL', 0.0), ('Phoenix, AZ', 0.0), ('Longview, TX', 0.030303030303030304), ('Great Falls, MT', 0.05714285714285714), ('Fresno, CA', 0.012195121951219513), ('Tucson, AZ', 0.034482758620689655), ('Moses Lake, WA', 0.1


### <strong> Bonus - Data Analysis </strong>
#### Data analysis

<p align="justify">
<font size="3">
Look at the number of almost empty flights and full flights and propose an analysis that explores if there is a correlation between the number of flights connecting two cities, the number of full flights and the percentage of almost empty flights. 

The granularity of the analysis is important: consider for example the week, the month and the year.
</font>
</p>


In [163]:
# Define here your considerations and your code